In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import nipy
import nibabel
import pandas as pd
import csv
import re
import os
from whatIHave import whatIHave


### TRAER DATOS DESDE CARPETAS FREESURFER/ CARPETA "STATS", BARRIENDO SUJETOS AUTOMÁTICAMENTE

In [2]:
#from nipype.interfaces.ants import CompositeTransformUtil

In [3]:
## Tupla con las tablas básicas estandard por hemisferio desde Freesurfer

basicStats=(
'lh.aparc.a2009s.stats',
'lh.aparc.DKTatlas.stats',
'lh.aparc.pial.stats',
'lh.aparc.stats',
'lh.BA_exvivo.stats',
'lh.BA_exvivo.thresh.stats',
'rh.aparc.a2009s.stats',
'rh.aparc.DKTatlas.stats',
'rh.aparc.pial.stats',
'rh.aparc.stats',
'rh.BA_exvivo.stats',
'rh.BA_exvivo.thresh.stats',
)

rutaStats='/home/pedro/Downloads/DataTesis'
#rutaStats='/home/pedro/Documents/freesurfer/freesurfer/subjects/'

In [4]:
# Se le entrega un path de archivo y genera una tupla con valores de la 
# tabla y sus subíndices fila, columna

def readTable(eachFile):
	fileOpen=eachFile
	f = open(fileOpen, 'r')
	indicador=open(fileOpen, 'r').read().find('ColHeaders')
	text=f.read()
	# Genera un nuevo texto desde original, desde el identificador de tablas
	sizeText=len(text)
	newText= text[indicador:sizeText]
	#Divide el nuevo texto en renglones
	reglones=newText.split('\n')
    
	p=re.compile(r'[\w+\.*\&*]*[\-*\w*]+')
	#
	i=0
	for reglon in reglones: # toma cada renglón y aplica procesamiento# 
	    palabras = p.findall(reglon) ### Hacer match con lo que se encuentra en RegExp ( No espacios pero palabras y números###
	    ## Chequeo de lontigud por si hay cosas debajo de la tabla##
	    if i==0: # Criterio para tomar la lontigud del primer renglon como regla para el resto
	        palabras.remove('ColHeaders')
	        headerCol=palabras
	        wTable=len(palabras)
	    if wTable==len(palabras):## Comprueba si las demás filas tienen el mismo ancho del primer renglon
	        #pass
	        #print (palabras) # Imprime la lista con las palabras separadas en un renglón
	        ## fin de chequeo de lontigud por si hay cosas debajo de la tabla##
	        j=0     ##### Equivalente a número de columna ######
	        ##Se hace un loop por cada renglon, para sacar las palabras y generar subíndices i j (fila, columna)
	        for palabra in palabras:#palabra corresponde a cada uno de los campos de la tabla
	            if j==0:
	                hName=re.sub(r'_', '-', palabra)# ahora ya no es la simple palabra, pero la palabra modificada de _ a -
	            hColF=(headerCol)[j]
	            if i>0 and j!=0:
	                yield (hName,hColF,palabra,i,j) #Imprime la palabra y el subíndice que identifica la posición real en la matriz origen
	            j+= 1 # Incrementa la columna
	        i+= 1   ##### Incrementa el número de renglón o fila #######

In [5]:
    ## Función para encontrar los archivos y el sujeto y poder llegar a leerlos 
    ## basado en ruta
    ## En donde busca dichos sujetos y Carpetas

def searchFS(basicStats, rutaStats ): 
	stats=re.compile(r'stats')
	nameSubjectR=re.compile(r'(\w+)\/stats')# RegExp para sacar el nombre del sujeto en el grupo en paréntesis

	for dirpath, dirs, files in os.walk(rutaStats): # Itera en directori rutaStats para buscar 
	    if stats.search(dirpath): #Busca 'stats' en dirpath
	        nameSubject=nameSubjectR.search(dirpath).group(1) #Toma el Reg Exp para sacar el nombre del sujeto ( Alfanumérico antes de '/stats')
	        for eachFile in files: # Itera archivos en el listado 'files' dentro del directorio que ya hizo match
	            if eachFile in basicStats:# Itera sobre cada archivo dentro de los directorios que hicieron match 'Stats' en este caso y contrasta contra la lista 'basicStats' que tiene el nombre de los archivos que queremos
	                for linea in readTable(dirpath+'/'+eachFile):# A readTable se le alimenta con la ruta y el archivo a analizar
	                    yield (eachFile,nameSubject,linea[0],linea[1],linea[2],linea[3],linea[4])# Itera sobre la función que genera los datos de tabla
                        ###### Nombre_Tabla[0], Sujeto[1], Parte[2], Feature[3], Valor[4] ######
	                #
	                #
	                #readTable(dirpath+'/'+eachFile)
	                #print (palabra,i,j)
	                #print (eachFile, nameSubject) # Imprime el archivo que hizo match y el sujeto de regExp
	                #print(dirpath+'/'+eachFile)
	                #f=open(dirpath+'/'+eachFile)
	                #text=f.read()
	                #print(text)
	                #break
    
   

In [6]:
##Solo funciona para las tablas con lh/rhxxxx.stats###for elemento in searchFS(basicStats, rutaStats):

## CUIDADO CON QUE REG EXP SE USA Y CUAL GRUPO DEBE HACER MATCH PARA BUSCAR
## LO QUE SE QUIERE ENCONTRAR

#nombre=re.compile(r'^(\wh)([\.*\w*\.]*)(stats)')
nombre=re.compile(r'^(\wh)(\.)([\.*\w*\.]*)(\.)(stats)')
##
#1= hemisferio
#3=nombre
#5=stats
##
arr = pd.DataFrame()
#palabras = nombre.findall(reglon)
##################################################################
## (iteración sobre la función anterior searchFS)
## (Imprime cada elemento de dicha fución que con 'yield', va produciendo ouput)

for elemento in searchFS(basicStats, rutaStats):
    tableN = nombre.match(elemento[0]).group(3)
    hemis = nombre.match(elemento[0]).group(1)
    key1=elemento[2]+'-'+elemento[3]+'-'+hemis+'-'+tableN #Elemento 2 es nombre del nucleo
    #    arr.at['Ftr: '+key1+','+hemis+','+elemento[2]+','+elemento[3], 'Sub: '+elemento[1]] = elemento[4]###Se llena el pandas array
    arr.at['Feature: '+key1+',Hemisphere: '+hemis+',Geometric: '+elemento[3]+',Nucleus: '+elemento[2], 'Sub: '+elemento[1]] = elemento[4]###Se llena el pandas array
    


In [7]:
arr.head()

,Sub: 940,Sub: 8,Sub: 145,Sub: 35,Sub: 694,Sub: 795,Sub: 195,Sub: 75,Sub: 1076
"Feature: G-and-S-frontomargin-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-frontomargin",927,1032,926,1103,1019,1048,1043,747,1105
"Feature: G-and-S-frontomargin-SurfArea-rh-aparc.a2009s,Hemisphere: rh,Geometric: SurfArea,Nucleus: G-and-S-frontomargin",670,741,649,759,711,705,719,559,825
"Feature: G-and-S-frontomargin-GrayVol-rh-aparc.a2009s,Hemisphere: rh,Geometric: GrayVol,Nucleus: G-and-S-frontomargin",1673,2115,1640,1955,2310,1623,2029,1515,2023
"Feature: G-and-S-frontomargin-ThickAvg-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickAvg,Nucleus: G-and-S-frontomargin",1.984,2.319,1.998,1.964,2.593,1.876,2.242,2.193,2.039
"Feature: G-and-S-frontomargin-ThickStd-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickStd,Nucleus: G-and-S-frontomargin",0.611,0.604,0.646,0.686,0.701,0.657,0.640,0.455,0.695


In [8]:
arr.to_csv('tSubjects.csv')

### CAPA DE ADAPTACIÓN PARA COOMPARACIÓN DE DATOS EN MISMA ESCALA. DATOS A COMPARAR DEBEN ESTAR EN FILAS Y NO EN COLUMNAS Y POR LO TANTO ES POSIBLEMENTE NECESARIO TRANSPONER


In [9]:
    # Read data from file 'filename.csv' 
    # (in the same directory that your python process is based)
    # Control delimiters, rows, column names with read_csv (see later) 
    #data = pd.read_csv("/home/pedro/mindboggle/ejemplos/osfstorage-archive/output/mindboggled/arno/tables/left_cortical_surface/label_shapes.csv", index_col = 'ID') 
    datain = pd.read_csv("tSubjects.csv", index_col = 0) 
    # Preview the first 5 lines of the loaded data 
    datain.head()

,Sub: 940,Sub: 8,Sub: 145,Sub: 35,Sub: 694,Sub: 795,Sub: 195,Sub: 75,Sub: 1076
"Feature: G-and-S-frontomargin-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-frontomargin",927.000,1032.000,926.000,1103.000,1019.000,1048.000,1043.000,747.000,1105.000
"Feature: G-and-S-frontomargin-SurfArea-rh-aparc.a2009s,Hemisphere: rh,Geometric: SurfArea,Nucleus: G-and-S-frontomargin",670.000,741.000,649.000,759.000,711.000,705.000,719.000,559.000,825.000
"Feature: G-and-S-frontomargin-GrayVol-rh-aparc.a2009s,Hemisphere: rh,Geometric: GrayVol,Nucleus: G-and-S-frontomargin",1673.000,2115.000,1640.000,1955.000,2310.000,1623.000,2029.000,1515.000,2023.000
"Feature: G-and-S-frontomargin-ThickAvg-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickAvg,Nucleus: G-and-S-frontomargin",1.984,2.319,1.998,1.964,2.593,1.876,2.242,2.193,2.039
"Feature: G-and-S-frontomargin-ThickStd-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickStd,Nucleus: G-and-S-frontomargin",0.611,0.604,0.646,0.686,0.701,0.657,0.640,0.455,0.695


In [10]:
data=datain.transpose()

In [11]:
## Solo si quiero  guardar la transposición
#data.to_csv('dataParallel.csv')

In [12]:
## Marcar la data que contiene cero con valor NAN para sacarla de los cómputos
#dNoCero=data[data != 0]

In [222]:
#dNoCero.describe()

In [13]:
#Calcula el valor medio de cada una de las variables numéricas

#Recibe un dataSet Pandas y genera impresión de los puntos medios de cada Columna numérica
#Recorre Columna por Columna en el Dataset
#lambda x: abs((x*10)/cMax)) if x>pMedio else -(x*10)/abs(cMax)
#dNoCero=data[data != 0]
estandarData = data.copy()

i=0
while i<len(data.columns):
    nameCol=data.columns[i]
    if data.dtypes[i]=='float64':
        cMax=float(data[nameCol].max())
        cMin=float(data[nameCol].min())
        pMedio=cMin+((cMax-cMin)/2)
        puntoMax=cMax-pMedio
        #print (pMedio,puntoMax)
        estandarData[nameCol] = data[nameCol].apply(lambda x: abs(((x-pMedio)*10)/puntoMax) if x>=pMedio else -abs(((x-pMedio)*10)/puntoMax))
    i += 1
    

In [14]:
#Calcula el valor medio de cada una de las variables numéricas

#Recibe un dataSet Pandas y genera impresión de los puntos medios de cada Columna numérica
#Recorre Columna por Columna en el Dataset
#lambda x: abs((x*10)/cMax)) if x>pMedio else -(x*10)/abs(cMax)
#dNoCero=data[data != 0]
estandarData = data.copy()

i=0
while i<len(data.columns):
    nameCol=data.columns[i]
    if data.dtypes[i]=='float64':
        cMax=float(data[nameCol].max())
        cMin=float(data[nameCol].min())
        pMedio=float(data[nameCol].mean())
        puntoMax=abs(cMax-pMedio)
        puntoMin=abs(pMedio-cMin)
        #print (pMedio,puntoMax)
        if puntoMax>=puntoMin:
            estandarData[nameCol] = data[nameCol].apply(lambda x: abs(((x-pMedio)*10)/puntoMax) if x>=pMedio else -abs(((x-pMedio)*10)/puntoMax))               
        else:
            estandarData[nameCol] = data[nameCol].apply(lambda x: abs(((x-pMedio)*10)/puntoMin) if x>=pMedio else -abs(((x-pMedio)*10)/puntoMin))                   
    i += 1
    
    


In [15]:
estandarData

,"Feature: G-and-S-frontomargin-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-SurfArea-rh-aparc.a2009s,Hemisphere: rh,Geometric: SurfArea,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-GrayVol-rh-aparc.a2009s,Hemisphere: rh,Geometric: GrayVol,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-ThickAvg-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickAvg,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-ThickStd-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickStd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-MeanCurv-rh-aparc.a2009s,Hemisphere: rh,Geometric: MeanCurv,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-GausCurv-rh-aparc.a2009s,Hemisphere: rh,Geometric: GausCurv,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-FoldInd-rh-aparc.a2009s,Hemisphere: rh,Geometric: FoldInd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-CurvInd-rh-aparc.a2009s,Hemisphere: rh,Geometric: CurvInd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-occipital-inf-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-occipital-inf",...,"Feature: transversetemporal-CurvInd-rh-aparc,Hemisphere: rh,Geometric: CurvInd,Nucleus: transversetemporal","Feature: insula-NumVert-rh-aparc,Hemisphere: rh,Geometric: NumVert,Nucleus: insula","Feature: insula-SurfArea-rh-aparc,Hemisphere: rh,Geometric: SurfArea,Nucleus: insula","Feature: insula-GrayVol-rh-aparc,Hemisphere: rh,Geometric: GrayVol,Nucleus: insula","Feature: insula-ThickAvg-rh-aparc,Hemisphere: rh,Geometric: ThickAvg,Nucleus: insula","Feature: insula-ThickStd-rh-aparc,Hemisphere: rh,Geometric: ThickStd,Nucleus: insula","Feature: insula-MeanCurv-rh-aparc,Hemisphere: rh,Geometric: MeanCurv,Nucleus: insula","Feature: insula-GausCurv-rh-aparc,Hemisphere: rh,Geometric: GausCurv,Nucleus: insula","Feature: insula-FoldInd-rh-aparc,Hemisphere: rh,Geometric: FoldInd,Nucleus: insula","Feature: insula-CurvInd-rh-aparc,Hemisphere: rh,Geometric: CurvInd,Nucleus: insula"
Sub: 940,-2.725640,-2.356542,-4.673663,-3.274401,-1.22500,1.824818,-0.740741,0.357143,-1.052632,3.203713,...,-5.00,-0.102555,1.890694,4.812427,1.929619,3.634424,10.000000,10.000000,-0.942833,6.527817
Sub: 8,1.517737,2.532517,5.508062,4.027610,-1.61875,5.109489,-0.740741,-1.428571,-1.052632,6.409935,...,3.75,2.297222,2.129985,-0.093787,-0.868035,-10.000000,-3.899614,-4.095948,-2.901024,-2.435700
Sub: 145,-2.766053,-3.802601,-5.433837,-2.969242,0.74375,-0.802920,0.000000,-1.785714,-2.631579,-6.121425,...,1.25,10.000000,10.000000,10.000000,3.460411,6.386913,-4.594595,8.984076,10.000000,10.000000
Sub: 35,4.387068,3.771997,1.822370,-3.710341,2.99375,-3.430657,-4.444444,-0.714286,-1.052632,-3.886101,...,2.50,0.199515,-1.166913,2.535170,9.900293,-2.190612,-2.509653,-3.896392,-2.843430,-2.322477
Sub: 694,0.992366,0.466718,10.000000,10.000000,3.83750,5.766423,10.000000,10.000000,10.000000,-6.618164,...,-1.25,-2.082789,-1.911374,-2.054513,2.985337,0.305832,-2.509653,-4.005241,-3.188993,-2.404249
Sub: 795,2.164347,0.053558,-5.825442,-5.628481,1.36250,-5.401460,1.851852,-0.357143,3.684211,-9.485700,...,10.00,-6.110386,-6.457903,-7.875147,-8.997067,1.201991,0.617761,-3.243298,-2.133106,-2.397959
Sub: 195,1.962281,1.017598,3.527003,2.349237,0.40625,-10.000000,-7.407407,-2.142857,-5.789474,10.000000,...,-2.50,4.025732,3.459380,3.449590,1.771261,-3.662873,-0.424710,3.469059,7.389078,-2.253285
Sub: 75,-10.000000,-10.000000,-8.313284,1.281182,-10.00000,2.481752,-3.333333,-2.857143,-8.947368,-2.802308,...,-3.75,-1.797501,-1.751846,-1.720399,-0.181818,3.250356,3.745174,-3.406571,-2.440273,-2.291026
Sub: 1076,4.467894,8.316756,3.388789,-2.075563,3.50000,4.452555,4.814815,-1.071429,6.842105,9.300050,...,-5.00,-6.429237,-6.192024,-9.053341,-10.000000,1.073969,-0.424710,-3.805684,-2.939420,-2.423120


In [16]:
estandarData.describe()

,"Feature: G-and-S-frontomargin-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-SurfArea-rh-aparc.a2009s,Hemisphere: rh,Geometric: SurfArea,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-GrayVol-rh-aparc.a2009s,Hemisphere: rh,Geometric: GrayVol,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-ThickAvg-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickAvg,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-ThickStd-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickStd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-MeanCurv-rh-aparc.a2009s,Hemisphere: rh,Geometric: MeanCurv,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-GausCurv-rh-aparc.a2009s,Hemisphere: rh,Geometric: GausCurv,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-FoldInd-rh-aparc.a2009s,Hemisphere: rh,Geometric: FoldInd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-frontomargin-CurvInd-rh-aparc.a2009s,Hemisphere: rh,Geometric: CurvInd,Nucleus: G-and-S-frontomargin","Feature: G-and-S-occipital-inf-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-occipital-inf",...,"Feature: transversetemporal-CurvInd-rh-aparc,Hemisphere: rh,Geometric: CurvInd,Nucleus: transversetemporal","Feature: insula-NumVert-rh-aparc,Hemisphere: rh,Geometric: NumVert,Nucleus: insula","Feature: insula-SurfArea-rh-aparc,Hemisphere: rh,Geometric: SurfArea,Nucleus: insula","Feature: insula-GrayVol-rh-aparc,Hemisphere: rh,Geometric: GrayVol,Nucleus: insula","Feature: insula-ThickAvg-rh-aparc,Hemisphere: rh,Geometric: ThickAvg,Nucleus: insula","Feature: insula-ThickStd-rh-aparc,Hemisphere: rh,Geometric: ThickStd,Nucleus: insula","Feature: insula-MeanCurv-rh-aparc,Hemisphere: rh,Geometric: MeanCurv,Nucleus: insula","Feature: insula-GausCurv-rh-aparc,Hemisphere: rh,Geometric: GausCurv,Nucleus: insula","Feature: insula-FoldInd-rh-aparc,Hemisphere: rh,Geometric: FoldInd,Nucleus: insula","Feature: insula-CurvInd-rh-aparc,Hemisphere: rh,Geometric: CurvInd,Nucleus: insula"
count,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,...,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00,9.000000e+00
mean,-2.960595e-16,3.552714e-15,-4.440892e-16,4.934325e-17,-1.036208e-15,-7.993606e-15,6.908054e-16,-2.467162e-17,-3.059281e-15,1.973730e-16,...,1.578984e-15,-3.355341e-15,-8.881784e-16,-2.763222e-15,5.329071e-15,-2.343804e-15,-5.865678e-15,-1.233581e-15,1.480297e-16,2.467162e-16
std,4.556751e+00,5.136625e+00,6.244994e+00,4.907577e+00,4.222478e+00,5.364964e+00,5.165306e+00,3.871336e+00,5.960396e+00,7.348870e+00,...,4.921255e+00,5.096261e+00,5.101575e+00,6.048313e+00,6.199769e+00,4.814998e+00,4.523311e+00,5.888076e+00,5.016266e+00,4.765333e+00
min,-1.000000e+01,-1.000000e+01,-8.313284e+00,-5.628481e+00,-1.000000e+01,-1.000000e+01,-7.407407e+00,-2.857143e+00,-8.947368e+00,-9.485700e+00,...,-5.000000e+00,-6.429237e+00,-6.457903e+00,-9.053341e+00,-1.000000e+01,-1.000000e+01,-4.594595e+00,-4.095948e+00,-3.188993e+00,-2.435700e+00
25%,-2.725640e+00,-2.356542e+00,-5.433837e+00,-3.274401e+00,-1.225000e+00,-3.430657e+00,-3.333333e+00,-1.785714e+00,-2.631579e+00,-6.121425e+00,...,-3.750000e+00,-2.082789e+00,-1.911374e+00,-2.054513e+00,-8.680352e-01,-2.190612e+00,-2.509653e+00,-3.896392e+00,-2.901024e+00,-2.404249e+00
50%,1.517737e+00,4.667177e-01,1.822370e+00,-2.075563e+00,7.437500e-01,1.824818e+00,-7.407407e-01,-1.071429e+00,-1.052632e+00,-2.802308e+00,...,-1.250000e+00,-1.025545e-01,-1.166913e+00,-9.378664e-02,1.771261e+00,1.073969e+00,-4.247104e-01,-3.406571e+00,-2.440273e+00,-2.322477e+00
75%,2.164347e+00,2.532517e+00,3.527003e+00,2.349237e+00,2.993750e+00,4.452555e+00,1.851852e+00,-3.571429e-01,3.684211e+00,6.409935e+00,...,2.500000e+00,2.297222e+00,2.129985e+00,3.449590e+00,2.985337e+00,3.250356e+00,6.1776

In [17]:
edo=estandarData.transpose()

In [18]:
edo.head()

,Sub: 940,Sub: 8,Sub: 145,Sub: 35,Sub: 694,Sub: 795,Sub: 195,Sub: 75,Sub: 1076
"Feature: G-and-S-frontomargin-NumVert-rh-aparc.a2009s,Hemisphere: rh,Geometric: NumVert,Nucleus: G-and-S-frontomargin",-2.725640,1.517737,-2.766053,4.387068,0.992366,2.164347,1.962281,-10.000000,4.467894
"Feature: G-and-S-frontomargin-SurfArea-rh-aparc.a2009s,Hemisphere: rh,Geometric: SurfArea,Nucleus: G-and-S-frontomargin",-2.356542,2.532517,-3.802601,3.771997,0.466718,0.053558,1.017598,-10.000000,8.316756
"Feature: G-and-S-frontomargin-GrayVol-rh-aparc.a2009s,Hemisphere: rh,Geometric: GrayVol,Nucleus: G-and-S-frontomargin",-4.673663,5.508062,-5.433837,1.822370,10.000000,-5.825442,3.527003,-8.313284,3.388789
"Feature: G-and-S-frontomargin-ThickAvg-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickAvg,Nucleus: G-and-S-frontomargin",-3.274401,4.027610,-2.969242,-3.710341,10.000000,-5.628481,2.349237,1.281182,-2.075563
"Feature: G-and-S-frontomargin-ThickStd-rh-aparc.a2009s,Hemisphere: rh,Geometric: ThickStd,Nucleus: G-and-S-frontomargin",-1.225000,-1.618750,0.743750,2.993750,3.837500,1.362500,0.406250,-10.000000,3.500000


In [16]:
#edoCol = edo.reset_index()
#edoColSep=edoCol['index'].str.split(pat = ',', expand=True)

In [17]:
#edo = pd.concat([edoColSep, edoCol], axis=1, sort=False)

In [18]:
#edo

In [19]:
edo.to_csv('normalizado.csv')

### DESDE TSV EN FORMATO TXT, SE PASAN DATOS A JSON PARA CLUSTERGRAMMER

In [20]:
## https://onlinecsvtools.com/convert-csv-to-tsv
#### INGRESAN ARCHIVOS TSV EN FORMATO TXT, NORMALMENTE DEBEN PRE CONVERTIRSE DESDE CSV

from clustergrammer import Network
net = Network()
net.load_file('toJson.txt')

/home/pedro/python3.5/p3.5/lib/python3.5/site-packages/clustergrammer/load_data.py:70: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  tmp_df['mat'] = pd.read_table(file_buffer, index_col=row_arr)


In [55]:
#anal = pd.read_csv("normalizado1.csv", index_col = 0)

In [56]:
#anal.columns

In [57]:
#anal['3'].unique()

In [21]:
## SE ALIMENTA CON TSV LUEGO DE SER CONVERTIDO

##Matriz de afinidad
##https://github.com/MaayanLab/clustergrammer-py/blob/master/make_clustergrammer.py
#The clustergrammer python module can be installed using pip:
#pip install clustergrammer
#or by getting the code from the repo:
#https://github.com/MaayanLab/clustergrammer-py

# from clustergrammer import Network
from clustergrammer import Network
net = Network()

# load matrix tsv file
#net.load_file('/home/pedro/Desktop/Codigo/Capa_Adaptacion/estandarData.tsv')
net.load_file('toJson.txt')
# net.load_file('txt/rc_val_cats.txt')

# optional filtering and normalization
##########################################
# net.filter_sum('row', threshold=20)
# net.normalize(axis='col', norm_type='zscore', keep_orig=True)
# net.filter_N_top('row', 250, rank_type='sum')
# net.filter_threshold('row', threshold=3.0, num_occur=4)
# net.swap_nan_for_zero()
# net.downsample(ds_type='kmeans', axis='col', num_samples=10)
# net.random_sample(random_state=100, num_samples=10, axis='col')
# net.clip(-6,6)
# net.filter_cat('row', 1, 'Gene Type: Interesting')
# net.set_cat_color('col', 1, 'Category: one', 'blue')

net.cluster(dist_type='cos',views=['N_row_sum', 'N_row_var'] , dendro=True,
             sim_mat=True, filter_sim=0.1, calc_cat_pval=False, enrichrgram=True)

# write jsons for front-end visualizations
net.write_json_to_file('viz', '/home/pedro/Desktop/cluster/clustergrammer-master/json/edo1.json', 'no-indent')
##net.write_json_to_file('sim_row', '/home/pedro/Desktop/cluster/clustergrammer-master/json/edo1_row.json', 'no-indent')
net.write_json_to_file('sim_col', '/home/pedro/Desktop/cluster/clustergrammer-master/json/edo1_col.json', 'no-indent')

in make_sim_mat, which_sim: ['row', 'col']


### TOMAR IMAGENES Y SEGMENTARLAS PARA LUEGO PASARLAS A FORMATO .OBJ

In [ ]:
from nipype.interfaces.ants import CompositeTransformUtil


In [ ]:
##Formato datos Kanguro
## Imágenes#########

###/home/pedro/Downloads/DataTesis/(35)_2019-03-15_18-33-16/35/mri/aparc+aseg.mgz
###/home/pedro/Downloads/DataTesis/(35)_2019-03-15_18-33-16/35/mri/wmparc.mgz
## Estadísticas#######

##/home/pedro/Downloads/DataTesis/(8)_2019-03-15_18-33-16/8/stats
##

In [ ]:
## wmparc, aparc+aseg, mejores resultados

#img = nibabel.load('wmparc.mgz')
#img = nibabel.load('/home/pedro/Documents/freesurfer/freesurfer/subjects/bert/mri/gmparc.mgz')
#img = nibabel.load('/home/pedro/Documents/freesurfer/freesurfer/subjects/bert/mri/brainmask.mgz')
#img = nibabel.load('/home/pedro/Documents/freesurfer/freesurfer/subjects/bert/mri/aseg.mgz')
#img = nibabel.load('/home/pedro/Documents/freesurfer/freesurfer/subjects/bert/mri/aparc+aseg.mgz')

#img = nibabel.load('/home/pedro/Downloads/DataTesis/(8)_2019-03-15_18-33-16/T1W3DTFESENSE.nii.gz')
img = nibabel.load('/home/pedro/Downloads/DataTesis/(35)_2019-03-15_18-33-16/35/mri/wmparc.mgz')


In [ ]:
image_data = img.get_fdata()
p=np.asarray(image_data)


In [ ]:
# Selecciona una sola seccion de un numpy array
section=p[:,100,:]
plt.imshow(section)
plt.show()

In [ ]:
# Se tiene el array 3d con todos los valores segmentados
k=np.unique(p)
len(k)

In [ ]:
for valores in k:
    whatIHave(valores) # Por cada valor ünico en el array, se entrega la equivalencia con la función whatIHave

In [ ]:
################## Segmentar


#mask=np.logical_not(np.logical_or(p==54, p==18))
mask=np.logical_or(p==17, p==53) # Mismo anterior, Mas simple?

np.place(p, mask, 100) # Pone 100 en el valor seleccionado por la máscara, es decir, reemplaza el valor
#np.place(p, p==17, 100)
np.place(p, p!=100, 0)

###################### Segmentar

In [ ]:
## SIMILAR AL ANTERIOR PERO TOMA CADA NÚMERO DE FORMA SEPARADA
##
np.place(p, p==17, 10) # Pone 100 en el valor seleccionado por la máscara
np.place(p, p==53, 10)
#np.place(p, p==17, 500)
#np.place(p, np.logical_and(p!=100, p!=500), 0)
np.place(p, p!=10, 0)

In [ ]:
#toma lo que hay en "valor" y lo resalta con un númerp 500, el resto de cerebro lo toma en 100
#np.place usa el array, mascara y valor a asignar
#valuePart= [7 or 8 or 16 or 46 or 47]
valuePart= [17 or 53]

mask=np.logical_and(p!=0, p!=valuePart)
np.place(p, mask, 100) # Pone 100 en el valor seleccionado por la máscara
np.place(p, p==valuePart, 500)

In [ ]:
np.unique(p)


In [ ]:
n=p

In [ ]:
p.sum()

In [ ]:
np.count_nonzero(p)

In [ ]:
from mayavi import mlab
src = mlab.pipeline.scalar_field(n)
mlab.pipeline.iso_surface(src, contours=[n.min()+0.1*n.ptp(), ], opacity=0.3)
mlab.pipeline.iso_surface(src, contours=[n.max()-0.1*n.ptp(), ],)

mlab.show()

In [ ]:
## Generar el archivo .obj desde el array "p"
##verts, faces, normals, values = measure.marching_cubes_lewiner(p, 0)
verts, faces, normals, values = measure.marching_cubes_lewiner(n, 0)
faces=faces +1

In [ ]:
thefile = open('hiposmall.obj', 'w')
for item in verts:
  thefile.write("v {0} {1} {2}\n".format(item[0],item[1],item[2]))

#for item in normals:
#  thefile.write("vn {0} {1} {2}\n".format(item[0],item[1],item[2]))

for item in faces:
  thefile.write("f {2}//{2} {1}//{1} {0}//{0}\n".format(item[0],item[1],item[2]))  

thefile.close()



##thefile.write("f {0}//{0} {1}//{1} {2}//{2}\n".format(item[0],item[1],item[2]))
##to
##thefile.write("f {2}//{2} {1}//{1} {0}//{0}\n".format(item[0],item[1],item[2]))